<img src="https://raw.githubusercontent.com/UXARRAY/uxarray/main/docs/_static/images/logos/uxarray_logo_h_dark.svg"
     width="30%"
     alt="HEALPix logo"
     align="right"
/>

# UXarray for Advanced HEALPix Analysis & Visualization

### In this section, you'll learn:

* Using the `uxarray` package to perform advanced analysis operators over HEALPix data such as non-conservative zonal means, etc.

### Related Documentation

* [UXarray homepage](https://uxarray.readthedocs.io/en/latest/index.html)
* [Working with HEALPix data - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/healpix.html)
* [UXarray overview - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/02-intro-to-uxarray/overview.html)
* [Data visualization with UXarray - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/03-plotting-with-uxarray/data-viz.html)
* [Cross-sections - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/cross-sections.html)
* [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [UXarray](https://uxarray.readthedocs.io/en/latest/index.html) | Necessary  | |
| [HEALPix overview](00-healpix) | Necessary  | |

**Time to learn**: 30 minutes

-----

In [ ]:
import cartopy.crs as ccrs
import holoviews as hv
import intake
import uxarray as ux

## Open data catalog

:::{tip}
We assume, you have already gone over the previous section, [UXarray for Basic HEALPix Statistics & Visualization](02-uxarray). If not and if you need to learn about data catalogs in general and the data we will use throughout this notebook, we recommend to check that section first.
:::

Let us open the online catalog from the [WCRP's Digital Earths Global Hackathon 2025](https://digital-earths-global-hackathon.github.io/) using `intake` and read the output of the `ICON` run `ngc4008`, which is stored in the HEALPix format:

In [ ]:
# Final data catalog location (once hackathon website (https://digital-earths-global-hackathon.github.io/) updated)
# cat_url='https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml'
# Interim data catalog location
cat_url = "https://raw.githubusercontent.com/digital-earths-global-hackathon/catalog/refs/heads/ncar/online/main.yaml"
cat = intake.open_catalog(cat_url)
model_run = cat.icon_ngc4008

We can look into a fine resolution dataset at **zoom level = 10** in it as `Xarray.Dataset`:

In [ ]:
ds = model_run(zoom=9, time="P1D").to_dask()
ds

### Create UXarray Datasets from HEALPix

We can use `from_healpix` as follows to open a HEALPix grid from `xarray.Dataset`:

In [ ]:
uxds = ux.UxDataset.from_healpix(ds)
uxds

### Data variable of interest

Then let us pick a variable from the dataset, which will give us an `uxarray.UxDataArray`:

In [ ]:
uxda = uxds["tas"]
uxda

### Global mean and plot

Computing the global air temperature mean (at the first timestep) and also having a quick plot of it would be a good idea to have as references to compare the upcoming analyses & visualizations to them:

In [ ]:
%%time
print(
    "Global air temperature average on ",
    uxda.time[0].values,
    ": ",
    uxda.isel(time=0).mean().values,
    " K",
)

In [ ]:
%%time
uxda.isel(time=0).plot(
    projection=ccrs.Robinson(),
    cmap="inferno",
    features=["borders", "coastline"],
    title="Global temperature",
    width=700,
)

## Rasterized Point Plots

When working with a higher-resolution dataset at a global scale, it's not always practical to render each cell as a polygon. Instead, we can rasterize the center of each pixel.

In [ ]:
# Controls the size of each pixel (smaller value leads to larger pixels)
pixel_ratio = 0.5

uxda.isel(time=0).plot.points(
    projection=ccrs.Robinson(),
    rasterize=True,
    dynamic=False,
    width=700,
    pixel_ratio=pixel_ratio,
    cmap="inferno",
    title=f"Global Air Temperature, pixel_ratio={pixel_ratio}",
)

If we decrease the size of each pixel (by setting the pixel ratio to a higher value), we can start to see missing values, which is due to a lower density of points near the poles, leading to some pixels not containing any of our original points.

Because of this, it's useful to try a few `pixel_ratio` values and see which one works best for your given resolution.

In [ ]:
# Controls the size of each pixel (smaller value leads to larger pixels)
pixel_ratio = 2.0

uxda.isel(time=0).plot.points(
    projection=ccrs.Robinson(),
    rasterize=True,
    dynamic=False,
    width=700,
    pixel_ratio=pixel_ratio,
    cmap="inferno",
    title=f"Global Air Temperature with bad pixel size selection, pixel_ratio={pixel_ratio}",
)

## Cross-Sections

UXarray provides several spatial cross-section methods:

- **Constant-Latitude Slice**
  Extract data along a fixed latitude.

- **Constant-Longitude Slice**
  Extract data along a fixed longitude.

- **Latitudinal-Interval Slice**
  Extract data between two latitude bounds.

- **Longitudinal-Interval Slice**
  Extract data between two longitude bounds.

To illustrate, we’ll use a zoom level of `4.


In [ ]:
uxda_coarse = ux.UxDataset.from_healpix(model_run(zoom=4, time="P1D").to_dask())["tas"]

### Constant Latitude

In [ ]:
boulder_lat = 40.0190

uxda_cross_lat = uxda_coarse.cross_section.constant_latitude(boulder_lat)

In [ ]:
uxda_cross_lat.isel(time=0).plot(
    rasterize=False,
    projection=ccrs.Robinson(),
    global_extent=True,
    cmap="inferno",
    features=["coastline"],
    title=f"Global temperature cross-section at {boulder_lat} degrees latitude",
    width=700,
    colorbar=False,
)

Since cross sections return a new instance of our data, we can perform operations directly on them.

In [ ]:
print(
    f"Mean at {boulder_lat} degrees lat (Boulder, CO, USA): {uxda_lat.mean().values} K"
)

### Latitude Interval

In [ ]:
uxda_lat_interval = uxda_coarse.cross_section.constant_latitude_interval(
    [boulder_lat - 15, boulder_lat + 15]
)

In [ ]:
uxda_lat_interval.isel(time=0).plot(
    rasterize=False,
    projection=ccrs.Robinson(),
    global_extent=True,
    cmap="inferno",
    clim=(220, 310),
    features=["coastline"],
    title=f"Global temperature cross-section at the latitude interval [{boulder_lat-5},{boulder_lat+5}] degrees",
    width=700,
    colorbar=False,
)

In [ ]:
print(
    f"Mean at the latitude interval of [{boulder_lat-5},{boulder_lat+5}] degrees (-/+15 degrees Boulder, CO, USA): {uxda_lat_interval.mean().values} K"
)

## Non-Conservative Zonal Mean

Calculating the zonal mean is easy by providing the latitude range between -90 and 90 degrees with a step size in degrees:

In [ ]:
zonal_mean_tas = uxda.isel(time=0).zonal_mean(lat=(-90, 90, 5))

In [ ]:
(
    uxda.isel(time=0).plot(
        cmap="inferno",
        # periodic_elements="split",
        height=300,
        width=600,
        colorbar=False,
        ylim=(-90, 90),
    )
    + zonal_mean_tas.plot.line(
        x="tas_zonal_mean",
        y="latitudes",
        height=300,
        width=180,
        ylabel="",
        ylim=(-90, 90),
        xlim=(220, 310),
        # xticks=[220, 250, 280, 310],
        yticks=[-90, -45, 0, 45, 90],
        grid=True,
    )
).opts(title="Temperature and its Zonal means at every 5 degrees latitude")

## Remapping & Custom Grid Topology

Now, let’s take a look at an example of UXarray’s remapping functionality. UXarray currently supports two remapping methods:

- **Nearest Neighbor**
  Assigns each target point the value of its closest source point.

- **Inverse Distance Weighted**
  Computes each target point’s value as the weighted average of nearby source points, with weights inversely proportional to distance.


For this example, we'll use the Nearest Neighbor approach.


### Region of Interest: Chicago

Before remapping, we restrict our data to a 2.5°×2.5° bounding box centered on Chicago, Illinois.

In [ ]:
chicago_lon = -87.6298
chicago_lat = 41.8781

degree_span = 2.5
half_span = degree_span / 2.0

Since we will be using this subset of data for remapping, let's make the bounding box slightly larger than the extent of our destination grid.

In [ ]:
degree_eps = 0.25
half_span_eps = half_span + degree_eps

healpix_subset = uxda.isel(time=0).subset.bounding_box(
    (chicago_lon - half_span_eps, chicago_lon + half_span_eps),
    (chicago_lat - half_span_eps, chicago_lat + half_span_eps),
)
healpix_subset.plot(
    title="Temperature Subset around Chicago (HEALPix zoom 9)",
    backend="matplotlib",
    width=500,
    height=500,
    aspect=2,
    cmap="inferno",
    features=["states", "rivers"],
)

### Creating our Destination Grid

UXarray wraps SciPy's mesh generation methods to support representing point cloud data build around the following algorithms
- Delaunay Triangulation
- Voronoi Tesselation

Let's create a grid of points centered around Chicago and triangulate it to create our destination grid.


In [ ]:
import numpy as np

n_points = 10

# Longitude Vector
lon_vals = np.linspace(
    chicago_lon - half_span, chicago_lon + half_span, n_points, dtype=np.float64
)

# Latitude Vector
lat_vals = np.linspace(
    chicago_lat - half_span, chicago_lat + half_span, n_points, dtype=np.float64
)

# Meshgrid
lon, lat = np.meshgrid(lon_vals, lat_vals)

# flatten for processing
lon_flat = lon.ravel()
lat_flat = lat.ravel()

# Create a mask of boundary points
min_lon, max_lon = lon_vals[0], lon_vals[-1]
min_lat, max_lat = lat_vals[0], lat_vals[-1]

mask = (
    np.isclose(lon_flat, min_lon)
    | np.isclose(lon_flat, max_lon)
    | np.isclose(lat_flat, min_lat)
    | np.isclose(lat_flat, max_lat)
)

boundary_points = np.flatnonzero(mask)

In [ ]:
destination_grid = ux.Grid.from_points(
    (lon_flat, lat_flat),
    method="regional_delaunay",
    boundary_points=boundary_points,
)

destination_grid.plot(
    title="Triangulated Points (Destination Grid)",
    backend="matplotlib",
    width=500,
    height=500,
    aspect=2,
    features=["states", "rivers"],
)

### Remapping

Now that we have both our subset of the original data and our custom destination grid, let’s remap!

In [ ]:
uxda_remapped = uxda.isel(time=0).remap.nearest_neighbor(
    destination_grid, coord_type="cartesian"
)

In [ ]:
(
    healpix_subset.plot(
        title="Temperature Subset around Chicago (HEALPix zoom 9)",
        backend="matplotlib",
        width=600,
        height=600,
        aspect=2,
        cmap="inferno",
        features=["states", "rivers"],
    )
    + uxda_remapped.plot(
        title="Remapped Temperature",
        backend="matplotlib",
        width=600,
        height=600,
        aspect=2,
        cmap="inferno",
        features=["states", "rivers"],
    )
).cols(1).opts(fig_size=150)